#  Analiza produkcji prądu w panelach fotowoltaicznych - sprawozdanie

Celem sprawozdania było zaimplementowanie najlepszego algorytmu eksploracji danych, który miał za zadanie przygotować model predykcyjny, pozwalający dowiedzieć się ile będzie produkowanego prądu w określonych momentach czasu w różnych lokalizacjach. Sprawozdanie zostało przygotowane na przedmiot - Zaawansowana Eksploracja Danych.

# Implementacja

Pierwszym krokiem jest import bibliotek oraz plików ze zbiorami testowymi oraz treningowymi.

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn import ensemble
from sklearn.preprocessing import MinMaxScaler


import pandas as pd
import numpy as np

df = pd.read_csv("elektrownie.csv")
df_test = pd.read_csv("test.csv")
submissions = pd.read_csv("sample_submission.csv")

Kolejnym krokiem jest wyczyszczenie danych z informacji, które nie będą przydatne w predykcji.

In [ ]:
df_filtered = df.drop(df.columns[31:48], axis=1)
df_filtered = df_filtered.drop(df_filtered.columns[20:28], axis=1)
df_filtered = df_filtered.drop(['id', 'idmodel', 'idbrand', 'lat', 'lon', 'ageinmonths', 'data', 'irri_pvgis_mod'], axis=1)

df_filtered_test = df_test.drop(df_test.columns[31:48], axis=1)
df_filtered_test = df_filtered_test.drop(df_filtered_test.columns[20:28], axis=1)
df_filtered_test = df_filtered_test.drop(['id',  'idmodel', 'idbrand', 'lat', 'lon', 'ageinmonths', 'data', 'irri_pvgis_mod'], axis=1)

W celu usunięcia nieprawidłowych danych zostaną usunięte informacje o próbkach, które mają zerowy napływ energii przy znaczącej ilości światła. W zbiorze danych jest również problem związany z tym, że od pewnego momentu po zakończeniu próbkowania, raz dziennie jest zapisywany ostatni pomiar.

In [ ]:
df_complete = df_filtered.dropna(how='any')
df_complete['ct'] = df_complete.groupby(['day', 'anno', 'idsito'])['ora'].transform('count')
df_cleaned = df_complete[df_complete['ct'] == 19]
df_cleaned = df_cleaned.drop(df_cleaned[(df_cleaned.kwh == 0) & (df_cleaned.irradiamento > 0.015)].index)

Podział na zbiory treningowy i testowy, ostateczny wybór parametrów, które są nieprzydatne podczas budowania modelu.

In [ ]:
X = df_cleaned.drop(['kwh', 'ct'], axis=1).drop(['idsito', 'anno', 'day', 'pressure'], axis=1)
y = df_cleaned.kwh
X_test = df_filtered_test.drop(['idsito', 'anno', 'day', 'pressure'], axis=1)
# print(X_test)

Korzystamy z regresora - GradientBoostingRegressor, aby znaleźć optymalne parametry posłużę się metodą GridSearchCV, która znajdzie optymalne parametry.

In [ ]:
param_grid = {'learning_rate': [0.1, 0.05, 0.02, 0.01],
                  'max_depth': [6, 8],
                  'min_samples_leaf': [ 9, 17, 29],
                  'max_features': [1.0, 0.3, 0.1] ## not possible in our example (only 1 
                  }

W celu zwiększenia poprawności klasyfikatora warto jest znormalizować dane.

In [ ]:
min_max=MinMaxScaler()

X_train = min_max.fit_transform(X)
X_test = min_max.fit_transform(X_test)

Walidacji modelu będę dokonywał korzystając z walidacji krzyżowej. W celu zapewnienia najlepszej jakości warto jest skorzystać z cross-walidacji opartej na szeregach czasowych, a nie na losowym doborze elementów.

In [ ]:
tscv = TimeSeriesSplit(n_splits=3)
my_cv = tscv.split(X_train)

Po przygotowaniu danych można ruszyć z budowaniem modelu. Wykorzystamy w tym celu GradientBoostingRegresor - jako parametr przyjmuje liczbę estymatorów które zostaną zbudowane w ramach treningu. Metodą GridSearchCV będziemy szukać najlepszego zbioru parametrów spośród wcześniej utworzonej tablicy param_grid. Verbose będzie nam wysyłać komunikaty związane z postępem poszukiwań, a n_jobs wskazuje ile wątków ma obsłużyć zadanie.

In [ ]:
est = ensemble.GradientBoostingRegressor(n_estimators=250)
gs_cv = GridSearchCV(est, param_grid, cv=my_cv, verbose=2, n_jobs=2).fit(X_train, y_train)

W celu zweryfikowania poprawności otrzymanego modelu możemy sprawdzić ich skuteczność, przy pomocy parametrów best_params_, best_estimator_, best_score_

In [ ]:
best_params = gs_cv.best_params_
model = gs_cv.best_estimator_
score = gs_cv.best_score_
# for item in gs_cv.grid_scores_:
    # print ("\t%s %s %s" % ('\tGRIDSCORES\t',  "R" , item))
# print ('%s\tHP\t%s\t%f' % ("R" , str(best_params) ,abs(score)))

Dokonujemy predykcji na podstawie danych testowych.

In [ ]:
y_pred = gs_cv.predict(X_test)

Otrzymane wyniki mają czasem ujemne wartości, aby to skorygować, stworzona została funkcja, która zeruje wartości ujemne.

In [ ]:
def val(i):
    if (i >= 0): return i
    else: return 0

Zapisujemy otrzymane wyniki do pliku, który może zostać umieszczony na platformie kaggle.

In [ ]:
y_pred = [ val(x) for x in y_pred ]
submissions['kwh'] = y_pred

# print(submissions)
submissions.to_csv('submission14.csv', index=False)